You are given an array of strings tokens that represents an arithmetic expression in a Reverse Polish Notation.

Evaluate the expression. Return an integer that represents the value of the expression.

Note that:

The valid operators are '+', '-', '*', and '/'.
Each operand may be an integer or another expression.
The division between two integers always truncates toward zero.
There will not be any division by zero.
The input represents a valid arithmetic expression in a reverse polish notation.
The answer and all the intermediate calculations can be represented in a 32-bit integer.
 

Example 1:

Input: tokens = ["2","1","+","3","*"]
Output: 9
Explanation: ((2 + 1) * 3) = 9
Example 2:

Input: tokens = ["4","13","5","/","+"]
Output: 6
Explanation: (4 + (13 / 5)) = 6
Example 3:

Input: tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"]
Output: 22
Explanation: ((10 * (6 / ((9 + 3) * -11))) + 17) + 5
= ((10 * (6 / (12 * -11))) + 17) + 5
= ((10 * (6 / -132)) + 17) + 5
= ((10 * 0) + 17) + 5
= (0 + 17) + 5
= 17 + 5
= 22
 

Constraints:

1 <= tokens.length <= 104
tokens[i] is either an operator: "+", "-", "*", or "/", or an integer in the range [-200, 200].

# brute force:
- ✅ Time Complexity: O(n²)
- Each time we scan the array to find the first operator (O(n)), and we reduce the list size by 2.
- We repeat this O(n) times in worst case → O(n²).

- ✅ Space Complexity: O(n) (or O(1) if you do it in-place)


In [8]:
class Solution:
    def evalRPN(self, tokens: list[str]) -> int:
        operations = {"+", "-", "*", "/"}

        while len(tokens) > 1:
            for i in range(len(tokens)):
                if tokens[i] in operations:
                    a = int(tokens[i - 2])
                    b = int(tokens[i - 1])
                    op = tokens[i]

                    if op == "+":
                        res = a + b
                    elif op == "-":
                        res = a - b
                    elif op == "*":
                        res = a * b
                    else:
                        res = int(a / b)  # truncate toward zero

                    # Replace tokens[i-2:i+1] with result
                    tokens = tokens[:i - 2] + [str(res)] + tokens[i + 1:]
                    print(tokens)
                    break  # start from beginning again

        return int(tokens[0])


In [9]:
Solution().evalRPN(tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"])

['10', '6', '12', '-11', '*', '/', '*', '17', '+', '5', '+']
['10', '6', '-132', '/', '*', '17', '+', '5', '+']
['10', '0', '*', '17', '+', '5', '+']
['0', '17', '+', '5', '+']
['17', '5', '+']
['22']


22

# approach 2:

- If it is a number the push it to stack.
- if it is a operator. then pop 2 elements from the stack adn do the operation and push it back.

In [ ]:
class Solution:
    def evalRPN(self, tokens: list[str]) -> int:
        operation_map = {"+": 0,
                         "-": 1,
                         "*": 2,
                         "/": 3}
        stack = []
        
        for ele in tokens:
            if ele not in operation_map:
                stack.append(ele)
            else:
                # take the 2 elements.
                # for / that should be in the given order.
                # ["4","13","5","/","+"]
                # so here, 13 / 5, so, first pop is b. ans second pop is a.
                b = stack.pop()
                a = stack.pop()
                operation = operation_map[ele]
                a, b = int(a), int(b)
                if operation == 0:
                    stack.append(a + b)
                elif operation == 1:
                    stack.append(a - b)
                elif operation == 2:
                    stack.append(a * b)
                else:
                    # division truncates toward 0
                    stack.append(int(a / b))
                    
        return int(stack[-1])

# tc:
# - we iterate over the list only once:
# - and the pop and push of stack, is O(1) only.
# tc - O(n)

# sc - O(n)

In [5]:
Solution().evalRPN(tokens = ["2","1","+","3","*"])

9

In [6]:
Solution().evalRPN(tokens = ["4","13","5","/","+"])

6

In [7]:
Solution().evalRPN(tokens = ["10","6","9","3","+","-11","*","/","*","17","+","5","+"])

22

In [ ]:
# rewrote using match-case
class Solution:
    def evalRPN(self, tokens: list[str]) -> int:
        operation_map = {"+": 0,
                         "-": 1,
                         "*": 2,
                         "/": 3}
        stack = []
        
        for ele in tokens:
            if ele not in operation_map:
                stack.append(ele)
            else:
                # take the 2 elements.
                # for / that should be in the given order.
                # ["4","13","5","/","+"]
                # so here, 13 / 5, so, first pop is b. ans second pop is a.
                b = stack.pop()
                a = stack.pop()
                operation = operation_map[ele]
                a, b = int(a), int(b)

                match operation:
                    case 0:
                        stack.append(a + b)
                    case 1:
                        stack.append(a - b)
                    case 2:
                        stack.append(a * b)
                    case 3:
                        # division truncates toward 0
                        stack.append(int(a / b))
                    
        return int(stack[-1])

# The division between two integers always truncates toward zero.

This line:

```python
stack.append(int(a / b))  # division truncates toward 0
```

is used to **mimic the behavior of integer division that truncates toward 0**, as required by the **LeetCode Reverse Polish Notation (RPN)** problem.

---

### ⚠️ Why is this needed?

In **C/C++ or Java**, when you do integer division:

```cpp
int res = -7 / 2;  // → result: -3 (truncates toward 0)
```

But in **Python**, when using the `//` operator:

```python
-7 // 2  # → result: -4  (rounds toward negative infinity, not zero!)
```

So LeetCode wants:

| `a` | `b` | Expected result |
| --- | --- | --------------- |
| 7   | 2   | 3               |
| -7  | 2   | -3 ✅ (toward 0) |
| 7   | -2  | -3 ✅ (toward 0) |
| -7  | -2  | 3               |

---

### ✅ How `int(a / b)` fixes it:

```python
int(a / b)
```

* `/` gives float division (e.g., `-7 / 2 = -3.5`)
* `int()` truncates toward zero (i.e., `int(-3.5) = -3`) ✅

So it handles **negative number division correctly**.

---

